In [1]:
import os
import sys
from dotenv import load_dotenv
from pathlib import Path
from datasets import load_dataset
import numpy as np
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import regex

if sys.platform == 'linux':
    load_dotenv(dotenv_path=Path('.') / '.env.linux')
elif sys.platform == 'win32':
    load_dotenv(dotenv_path=Path('.') / '.env.win')
else:
    raise ValueError('Ваша операционная система не поддерживается')

os.environ['HF_HOME'] = os.getenv('HUGGING_FACE_CACHE_DIR')
DATASET_FOLDER = os.getenv('DATASET_FOLDER_PATH', None)
DATASET_PATH = os.getenv('DATASET_PATH', None)
MODEL_NAME = os.getenv('MODEL_NAME', None)

In [ ]:
prepared_df = pd.read_csv(os.path.join(DATASET_FOLDER, 'all_recepies_preprocessed.csv')).rename(columns={'Instructions' : 'instructions'})

for i in range(len(prepared_df)):
    prepared_df.loc[i, 'name'] = prepared_df.loc[i, 'name'][0].upper() + prepared_df.loc[i, 'name'][1:]
    prepared_df.loc[i, 'name'] = regex.sub(r'^\n', '', prepared_df.loc[i, 'name'])
    prepared_df.loc[i, 'name'] = regex.sub(r'\n$', '', prepared_df.loc[i, 'name'])

    prepared_df.loc[i, 'instructions'] = regex.sub(r'\r', '', prepared_df.loc[i, 'instructions'])
    prepared_df.loc[i, 'instructions'] = regex.sub(r'^\n', '', prepared_df.loc[i, 'instructions'])
    prepared_df.loc[i, 'instructions'] = regex.sub(r'\n$', '', prepared_df.loc[i, 'instructions'])

    prepared_df.loc[i, 'ingredients'] = regex.sub(r' \.\n', '\n', prepared_df.loc[i, 'ingredients'])
    prepared_df.loc[i, 'ingredients'] = regex.sub(r'^\n', '', prepared_df.loc[i, 'ingredients'])
    prepared_df.loc[i, 'ingredients'] = regex.sub(r'\n$', '', prepared_df.loc[i, 'ingredients'])

prepared_df.to_csv(os.path.join(DATASET_FOLDER, 'all_recepies_preprocessed_fixed.csv'), index=False)

In [3]:
prepared_df = pd.read_csv(os.path.join(DATASET_FOLDER, 'all_recepies_preprocessed_fixed.csv'))

In [4]:
prepared_df.head(2)

,name,instructions,ingredients,composition_list
0,Рассольник классический с перловкой и солеными...,Подготовить указанные ингредиенты для приготов...,Перловка 0.1 стак.\n Соленые огурцы 0.4 шт.\n...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ..."
1,Суп пюре из белокочаной капусты,"Необходимые ингредиенты\nНарезаем лук, морковь...",Капуста белокочанная 50.0 гр.\n Картошка 30.0...,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},..."


In [5]:
def concat_features_in_one(row):
    return f'''Название рецепта: {row['name']}

Ингридиенты:
{row['ingredients']}

Процесс приготовления:
{row['instructions']}'''

prepared_df['prompt'] = prepared_df.apply(concat_features_in_one, axis=1)

In [6]:
prepared_df.to_csv(os.path.join(DATASET_FOLDER, 'fine_tuning_preprocessed.csv'), index=False)

## Исследуем данные для подбора оптимальных параметров токенизации

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [8]:
full_dataset = load_dataset('csv', data_files=[DATASET_PATH])
full_dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['name', 'instructions', 'ingredients', 'composition_list', 'prompt'],
        num_rows: 27884
    })
})

In [9]:
tokens_count = []
def find_tokens_count(row):
    length = tokenizer(row['prompt'], return_length=True)['length'][0]
    tokens_count.append(length)
    return {'length' : length}

full_dataset = full_dataset.map(find_tokens_count, batched=False)

Map:   0%|          | 0/27884 [00:00<?, ? examples/s]

In [10]:
all_lengthes_np = np.array(tokens_count)
print(f'Всего токенов = {all_lengthes_np.sum()/1000**2} миллионов')
print(f'Минимум токенов = {all_lengthes_np.min()}')
print(f'Максимум токенов = {all_lengthes_np.max()}')
print(f'Среднее количество токенов = {all_lengthes_np.mean()}')
print(f'Медиана = {np.median(all_lengthes_np)}')

quantile_val = 0.98
quantile = np.quantile(all_lengthes_np, quantile_val)
print(f'Квантиль {quantile_val} = {quantile}')

Всего токенов = 11.632212 миллионов
Минимум токенов = 81
Максимум токенов = 2388
Среднее количество токенов = 417.1643953521733
Медиана = 381.0
Квантиль 0.98 = 902.0


In [ ]:
# Отфильтруем слишком длинные предложения
filtered_dataset = full_dataset.filter(lambda x : x['length'] <= quantile, batched=True)

Filter:   0%|          | 0/27884 [00:00<?, ? examples/s]